In [83]:
from data import bAbIGen
import numpy as np
import pickle
from collections import namedtuple
from keras.utils.np_utils import to_categorical
import keras
import seaborn as sns

from model import MyModel

In [84]:
config = {
    "max_context_len": 500,
    "max_sentence_num": 100,
    "max_sentence_len": 15,
    "max_question_len": 15,
    "max_answer_len": 6,
    "vocab_size": 200,
    "hidden_dim": 80
}

config = namedtuple("Config", config.keys())(*config.values())

In [85]:
with open("vocab.pickle", "rb") as file:
    vocab, rev_vocab = pickle.load(file)

In [86]:
train_gen = bAbIGen("../mixed/train_all.txt", config).generate_batches(1000, vocab)
valid_gen = bAbIGen("../mixed/valid_all.txt", config).generate_batches(1000, vocab)

In [87]:
def gen_wrap(gen):
    for c, q, a in gen:
        yield [c, q], np.stack([to_categorical(ans, config.vocab_size) for ans in a])

In [ ]:
m = MyModel(config)

In [ ]:
m.model.fit_generator(
    gen_wrap(train_gen),
    100,
    100,
    validation_data=gen_wrap(valid_gen),
    validation_steps=20,
    callbacks=[
        keras.callbacks.ModelCheckpoint("best_loss", save_best_only=True),
        keras.callbacks.EarlyStopping(patience=200)
    ]
)

Epoch 1/100
100/100 [==============================] - 509s - loss: 1.8159 - acc: 0.6416 - val_loss: 1.2973 - val_acc: 0.6488